In [0]:
import openai 
from pyspark.sql import functions as F
import time

In [0]:
openai.api_type = 'azure'
openai.api_key = '************'
openai.api_base = 'https://openai*****.openai.azure.com/'
openai.api_version = "2023-05-15"

In [0]:
df_companies = spark.sql('''SELECT DISTINCT Customer_Name FROM industry_ci WHERE Industry_lvl1 = 'Industry Not Assigned' LIMIT 1000 ''')
                         

df_industries = spark.sql('''SELECT DISTINCT Industry_Group FROM sic_mapping WHERE Industry_Group != 'Non Classifiable' ''')
df_sub_industries = spark.sql('''SELECT DISTINCT SIC_Description FROM sic_mapping''')
df_topcompanies = spark.sql('''
WITH df AS (SELECT Customer_Name, SUM(total_amount_usd) 
FROM industry_bi_final_updated
WHERE Customer_Name NOT IN ('NO CLIENT', '-')
GROUP BY Customer_Name
ORDER BY SUM(total_amount_usd) DESC
LIMIT 100) SELECT DISTINCT Customer_Name FROM df                          
''')

In [0]:
company_list = df_companies.select('Customer_Name').rdd.flatMap(lambda x: x).collect()
industry_list = df_industries.select('JLL_Industry_Group').rdd.flatMap(lambda x: x).collect()
sub_industry_list = df_sub_industries.select('SIC_Description').rdd.flatMap(lambda x: x).collect()

In [0]:
df_topcompanies = df_topcompanies.withColumn("index", F.monotonically_increasing_id())

batch_size = 5
df_topcompanies = df_topcompanies.withColumn("batch_id", (df_topcompanies["index"] / batch_size).cast("integer"))

df_batches = df_topcompanies.groupby("batch_id").agg(F.collect_list("Customer_Name").alias("companies"))

responses = []

while True:
    try:
        if responses:
            start_index = len(responses)
        else:
            start_index = 0

        for row in df_batches.collect()[start_index:]:
            company_list_string = "\n".join(row["companies"])
            prompt_text = f'''   
Instructions: (
I want you to act as a financial analyst. 
Based on the given list of companies: {company_list_string} 
Please produce the following information in this format: !!CompanyA||IndustryA||ConfidenceScoreA|||CompanyB||IndustryB||ConfidenceScoreB|||CompanyC||IndustryC||ConfidenceScoreC!!:

1.Each company's corresponding industry by only choosing from this industry list:{industry_list}

2.the associated confidence score on a scale of 1-100)


Rules: (
1.When facing similar names, for example: Microsoft instead of Microsoft Corporation, or Amazon.com instead of Amazon assume they are the same company.
2.If a company has businesses in multiple industries, choose the industry that reflecs the company's base/core.
  For example, even tho facebook is a social media company, the way they make money is through technology. Or Amazon is a E-Commmerce comapny but they are really a technology company.
3.Adhere strictly to the prescribed format as one continuous string: for instance, the result should mimic this format: !!CompanyA||IndustryA||ConfidenceScoreA|||CompanyB||IndustryB||ConfidenceScoreB|||CompanyC||IndustryC||ConfidenceScoreC!!
'!!' is used to wrap the begin and end of the string.
4.Do NOT use any other formatting including but not limited to: indexes, indents, spaces, headers, new lines, empty results
5.Do NOT omit or truncate records, produce all results.)
'''

            prompt = [{"role": "user", "content": prompt_text}]
            response = openai.ChatCompletion.create(deployment_id='gpt-35-turbo-16k', engine="gpt-35-turbo-16k", messages=prompt, temperature = 0)
            
            gpt_answer = response["choices"][0].message.content.strip()
            
            companies_industries_scores = gpt_answer.strip().split('|||')
            
            for item in companies_industries_scores:
                item = item.replace('!!', '')
                try:
                    company, industry, confidence_score = map(str.strip, item.split('||'))
                    responses.append((company, industry, confidence_score))
                except ValueError as ve:
                    print(f"Error parsing item {item}: {ve}")

        header = ["Company", "Industry", "Confidence_Score"]
        sdf = spark.createDataFrame(data=responses, schema=header)
        sdf = sdf.withColumn("Confidence_Score", sdf["Confidence_Score"].cast("integer"))

        sdf.createOrReplaceTempView('GPT_topIndustry')
        break

    except Exception as e:  
        print(f"Error: {e}")
        time.sleep(3)

In [0]:
%sql
SELECT *
FROM GPT_topIndustry A
LEFT JOIN industry_ci B
ON A.Company = B.Customer_Name
WHERE A.Industry = B.Industry_lvl1

Company,Industry,Confidence_Score,BEST_CUST_ID,CompanyId,Customer_Name,Industry_lvl1,Mastered_Customer_Name,Mastered_Industry,SIC_CODE,SIC_Description
Bank of America,Financial Services,100,CUS0023731,C-00086,Bank of America,Financial Services,Bank of America,Financial Services,6021,National Commercial Banks
Bank of America,Financial Services,100,130158,C-00086,Bank of America,Financial Services,Bank of America,Financial Services,null,null
Bank of America,Financial Services,100,196604,C-00086,Bank of America,Financial Services,Bank of America,Financial Services,6021,National Commercial Banks
Bank of America,Financial Services,100,71833,C-00086,Bank of America,Financial Services,Bank of America,Financial Services,6021,National Commercial Banks
Bank of America,Financial Services,100,CUS0207816,C-00086,Bank of America,Financial Services,Bank of America,Financial Services,6021,National Commercial Banks
Bank of America,Financial Services,100,244992,C-00086,Bank of America,Financial Services,Bank of America,Financial Services,6021,National Commercial Banks
Bank of America,Financial Services,100,21229,C-00086,Bank of America,Financial Services,Bank of America,Financial Services,6021,National Commercial Banks
Bank of America,Financial Services,100,71743,C-00086,Bank of America,Financial Services,Bank of America,Financial Services,6021,National Commercial Banks
Bank of America,Financial Services,100,193206,C-00086,Bank of America,Financial Services,Bank of America,Financial Services,6021,National Commercial Banks
Bank of America,Financial Services,100,CUS0269465,C-00086,Bank of America,Financial Services,Bank of America,Financial Services,6021,National Commercial Banks


In [0]:
df_companies = df_companies.withColumn("index", F.monotonically_increasing_id())
batch_size = 100
df_companies = df_companies.withColumn("batch_id", (df_companies["index"] / batch_size).cast("integer"))
df_batches = df_companies.groupby("batch_id").agg(F.collect_list("Customer_Name").alias("companies"))

responses = []

while True:
    try:
        #If already at least one response exists, the process will continue after the last successfully processed batch
        if responses:
            start_index = len(responses)
        else:
            start_index = 0

        for row in df_batches.collect()[start_index:]:
            company_list_string = "\n".join(row["companies"])
            prompt_text = f'''   
Instructions: (
I want you to act as a financial analyst. 
Based on the given list of companies: {company_list_string} 
Please produce the following information in this format: !!CompanyA||IndustryA||ConfidenceScoreA|||CompanyB||IndustryB||ConfidenceScoreB|||CompanyC||IndustryC||ConfidenceScoreC!!:

1.Each company's corresponding industry by only choosing from this industry list:{industry_list}

2.the associated confidence score on a scale of 1-100)


Rules: (
1.When facing similar names, for example: Microsoft instead of Microsoft Corporation, or Amazon.com instead of Amazon assume they are the same company.
2.If a company has businesses in multiple industries, choose the industry that reflecs the company's base/core.
  For example, even tho facebook is a social media company, the way they make money is through technology. Or Amazon is a E-Commmerce comapny but they are really a technology company.
3.Adhere strictly to the prescribed format as one continuous string: for instance, the result should mimic this format: !!CompanyA||IndustryA||ConfidenceScoreA|||CompanyB||IndustryB||ConfidenceScoreB|||CompanyC||IndustryC||ConfidenceScoreC!!
'!!' is used to wrap the begin and end of the string.
4.Do NOT use any other formatting including but not limited to: indexes, indents, spaces, headers, new lines, empty results)
5.Do NOT omit or truncate records, produce all results.
'''

            prompt = [{"role": "user", "content": prompt_text}]
            response = openai.ChatCompletion.create(deployment_id='gpt-35-turbo-16k', engine="gpt-35-turbo-16k", messages=prompt, temperature = 0)
            
            gpt_answer = response["choices"][0].message.content.strip()
            
            companies_industries_scores = gpt_answer.strip().split('|||')
            
            for item in companies_industries_scores:
                item = item.replace('!!', '')
                try:
                    company, industry, confidence_score = map(str.strip, item.split('||'))
                    responses.append((company, industry, confidence_score))
                except ValueError as ve:
                    print(f"Error parsing item {item}: {ve}")

        header = ["Company", "Industry", "Confidence_Score"]
        sdf = spark.createDataFrame(data=responses, schema=header)
        sdf = sdf.withColumn("Confidence_Score", sdf["Confidence_Score"].cast("integer"))

        sdf.createOrReplaceTempView('GPT_Industry')
        break

    except Exception as e:  
        print(f"Error: {e}")
        time.sleep(3)

In [0]:
%sql

SELECT *
FROM GPT_Industry A
LEFT JOIN industry_ci B
ON A.Company = B.Customer_Name

Company,Industry,Confidence_Score,BEST_CUST_ID,CompanyId,Customer_Name,Industry_lvl1,Mastered_Customer_Name,Mastered_Industry,SIC_CODE,SIC_Description
Global HoldingsManagement Inc,Financial Services,100,273530,C-14394,Global HoldingsManagement Inc,Industry Not Assigned,Global HoldingsManagement Inc,Industry Not Assigned,null,null
Democratic Mayors Association,Government,100,CUS0363333,null,Democratic Mayors Association,Industry Not Assigned,Democratic Mayors Association,Industry Not Assigned,null,null
Gilde Brewery,Food and Beverages,100,CUS0059622,null,Gilde Brewery,Industry Not Assigned,Gilde Brewery,Industry Not Assigned,null,null
James J. Boyle,Professional Services,100,269491,null,James J. Boyle,Industry Not Assigned,James J. Boyle,Industry Not Assigned,null,null
Hasenstab Architects,Construction,100,CUS0304942,null,Hasenstab Architects,Industry Not Assigned,Hasenstab Architects,Industry Not Assigned,null,null
High Museum of Art,Non-Profit,100,CUS0244706,null,High Museum of Art,Industry Not Assigned,High Museum of Art,Industry Not Assigned,null,null
1226299 Alberta Ltd.,Real Estate,100,CUS0196654,null,1226299 Alberta Ltd.,Industry Not Assigned,1226299 Alberta Ltd.,Industry Not Assigned,null,null
Velez Organization,Real Estate,100,CUS0383047,null,Velez Organization,Industry Not Assigned,Velez Organization,Industry Not Assigned,null,null
Diane Currier,Professional Services,100,249352,null,Diane Currier,Industry Not Assigned,Diane Currier,Industry Not Assigned,null,null
"FanX, Inc. d.b.a Gameday VR",Media and Entertainment,100,CUS0359530,null,"FanX, Inc. d.b.a Gameday VR",Industry Not Assigned,"FanX, Inc. d.b.a Gameday VR",Industry Not Assigned,null,null


In [0]:
df_topcompanies = df_topcompanies.withColumn("index", F.monotonically_increasing_id())

batch_size = 50
df_topcompanies = df_topcompanies.withColumn("batch_id", (df_topcompanies["index"] / batch_size).cast("integer"))

df_batches = df_topcompanies.groupby("batch_id").agg(F.collect_list("Customer_Name").alias("companies"))

responses = []

for row in df_batches.collect():
    company_list_string = "\n".join(row["companies"])

    prompt_text = f'''   
Instructions: (
Based on the given list of companies: {company_list_string} 
Please:
1. classify whether they are in the forbes global 2000 list with a Yes/No
2. produce the company's head quarter, put unkown if null
3. produce the company's net worth





Rules: (
1.Make sure to classify all companies.
2.When facing similar names, for example: Microsoft instead of Microsoft Corporation, assume Microsoft is Microsoft Corporation.
3.Adhere strictly to the prescribed format as one continuous string: for instance, the result should mimic this format: "!!CompanyA||Yes/No||headquarterA||revenueA|||CompanyB||Yes/No||headquarterA||revenueA|||CompanyC||Yes/No||headquarterA||revenueA!!".
4.Do NOT use any other formatting including but not limited to: indexes, indents, spaces, headers, new lines.)
'''

    prompt = [{"role": "user", "content": prompt_text}]
    response = openai.ChatCompletion.create(deployment_id='gpt-35-turbo-16k', engine="gpt-35-turbo-16k", messages=prompt, temperature = 0)
    gpt_answer = response["choices"][0].message.content.strip()
    
    companies_industries_scores = gpt_answer.strip().split('|||')
    
    for item in companies_industries_scores:
        item = item.replace('!!', '')
        try:
            company, Forbes, headquarter, revenue = map(str.strip, item.split('||'))
            responses.append((company, Forbes, headquarter, revenue))
        except ValueError as ve:
            print(f"Error parsing item {item}: {ve}")

header = ["Company", "Forbes", "Headquarter", "Net Worth"]
sdf = spark.createDataFrame(data=responses, schema=header)
#sdf = sdf.withColumn("Confidence_Score", sdf["Confidence_Score"].cast("integer"))

sdf.createOrReplaceTempView('GPT_Forbes')

Error parsing item LBA Realty||No||Unknown||Unknown||: too many values to unpack (expected 4)


In [0]:
%sql

SELECT * FROM GPT_Forbes
WHERE Forbes = 'Yes'

Company,Forbes,Headquarter,Revenue
Bank of America,Yes,"Charlotte, North Carolina, United States",$354.9 billion
Google,Yes,"Mountain View, California, United States",$1.2 trillion
Amazon,Yes,"Seattle, Washington, United States",$1.7 trillion
Amazon.com,Yes,"Seattle, Washington, United States",$1.7 trillion
"Apple, Inc.",Yes,"Cupertino, California, United States",$2.3 trillion
UnitedHealth Group,Yes,"Minnetonka, Minnesota, United States",$387.1 billion
HP Inc.,Yes,"Palo Alto, California, United States",$35.2 billion
HSBC Holdings plc,Yes,"London, United Kingdom",$133.6 billion
JP MORGAN CHASE BANK NA,Yes,"New York, New York, United States",$3.4 trillion
"Amgen, Inc",Yes,"Thousand Oaks, California, United States",$138.2 billion
